# **Bismi Allah**

# **Projet d'ARN de fin de semestre**


        Travail réalisé par le binôme : 
            - SAYOUD Maissa 191931040670
            - BOULKABOUL Amira 202031043294

    Etudiantes en M1 SII Groupe 02

            


## **Etape 0: Importation des Bibliothèques & Chargement des Données**

In [71]:
# pour mettre les données dans un dataset pandas dataframe
import pandas as pd

# pour generer des nombre aleatoires
import random

# pour les expression régulieres (regex)
import re

# pour la vectorisation
import numpy as np

#pour les chaines de caracteres
import string

# pour supprimer les stop words
import nltk
from nltk.corpus import stopwords
",".join(stopwords.words('english'))
stop_words=set(stopwords.words('english'))
from nltk.tokenize import word_tokenize

# pour la tokenisation 
nltk.download('punkt')
nltk.download('wordnet')
import spacy

# pour le stemming
from nltk.stem import PorterStemmer

# Pour lemmatisation 
from nltk.stem import WordNetLemmatizer



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


'import matplotlib.pyplot as plt\nimport cv2\n\nfrom sklearn.metrics import log_loss\nfrom sklearn.neural_network import MLPClassifier\n\nimport nltk\nfrom nltk.corpus import stopwords\nfrom nltk.tokenize import word_tokenize\nfrom nltk.stem import PorterStemmer\nfrom bs4 import BeautifulSoup'

## Chargement des Données du Dataset "sentiment140"
Pour jeter un coup d'oeil à des exemples de l'ensemble de données à traiter !

In [41]:

dataset = pd.read_csv("training.1600000.processed.noemoticon.csv", 
                      header=None, 
                      encoding="ISO-8859-1",
                      names=["target", "ids", "date", "flag", "user", "text"]
                      )

print(dataset.shape)



(1600000, 6)


In [42]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 6 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   target  1600000 non-null  int64 
 1   ids     1600000 non-null  int64 
 2   date    1600000 non-null  object
 3   flag    1600000 non-null  object
 4   user    1600000 non-null  object
 5   text    1600000 non-null  object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB


## Visualisation de 3 données aléatoires du Dataset "sentiment140"

In [43]:

nombre_tweets = 3

r = random.sample(range(len(dataset)), nombre_tweets)

for i in r:
    print(" Visualiser le tweet N° : ", i+1)
    
    print("target : ",dataset.iloc[i]["target"])
    print("ids  : ",dataset.iloc[i]["ids"])
    print("date : ",dataset.iloc[i]["date"])
    print("flag : ",dataset.iloc[i]["flag"])
    print("user : ",dataset.iloc[i]["user"])
    print("text : ",dataset.iloc[i]["text"])
    print()



 Visualiser le tweet N° :  1572803
target :  4
ids  :  2188988098
date :  Mon Jun 15 22:59:54 PDT 2009
flag :  NO_QUERY
user :  zdnetaustralia
text :  We couldn't resist this little headline fun on Superwoman   http://bit.ly/Ooe61 *chortle*

 Visualiser le tweet N° :  136293
target :  0
ids  :  1880065846
date :  Thu May 21 23:59:47 PDT 2009
flag :  NO_QUERY
user :  MZenani
text :  @ilikecupcakes that?sucks balls. I'm sorry. A Toyota clipped my back wheel when I was changing lanes on the 105 and he sped up to stop me 

 Visualiser le tweet N° :  418155
target :  0
ids  :  2061701792
date :  Sat Jun 06 21:25:37 PDT 2009
flag :  NO_QUERY
user :  cmrush
text :  @willychu hardware error. People couldn't hear me speaking through the phone.  PS eating @ asuka ramen... So yummy.



# **Étape 1 : Préparation des Données**




## **A. Nettoyage des Tweets** :

- **Supprression des colonnes inutiles**

- **Minuscule** : l'intégralité du tweet devra être convertie en minuscules.  

- **Suppression de balises HTML** : Toutes les balises HTML devront être supprimées.  

- **Radicalisation de mots** : Les mots devront être réduits à leur forme radicale.

- **Suppression des non-mots** : les non-mots et la ponctuation devront être supprimés. Tous les espaces blancs (onglets, nouvelles lignes, espaces) devront être remplacés par un seul espace. 

- **Suppression des mots vides** : les mots tel que « a, the, this,… » devront être supprimés.

- **Remplacement de label 4 par label 1**.

etc.



### **Suppression des colonnes inutiles**
Les colonnes comme "ids", "date", "flag" et "user" ne sont pas nécessaires pour
l'analyse de sentiment. Nous gardons que le texte des tweets!

In [44]:
# Supprimer les colonnes inutiles
dataset.drop(columns=["ids", "date", "flag", "user"], inplace=True)

print("New shape :", dataset.shape)


New shape : (1600000, 2)


### **Fonctions de Nettoyage des Données**


In [45]:
# fonction pour supprimer les URL
def remove_url(input_text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'', input_text)


In [46]:
# fonction pour supprimer les balises HTML:
def remove_html_tags(input_text):
    html = re.compile(r'<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
    return re.sub(html, '', input_text)


In [47]:
# fonction pour supprimer les tags (@user):
def remove_pattern(input_text, pattern):
    r = re.findall(pattern, input_text)
    for word in r:
        input_text = re.sub(word, "", input_text)
    return input_text

In [48]:
# fonction pour supprimer les non-words et la poctuation 
def remove_non_words(input_text):
    cleaned_text = re.sub(r'[^a-zA-Z]', ' ', input_text)
    
    return cleaned_text


In [49]:
# fonction pour supprimer les blancs:
def remove_gaps(input_text):
    texte_propre = re.sub(r'\s+', ' ', input_text)
    return texte_propre


In [50]:
# focntion pour supprimer les mots vides :
def remove_stop_words (input_text):
    return " ".join([ word for word in str(input_text).split() if word not in stop_words])


In [51]:
# fonction pour convertir les tweets en minuscule:
def lower_text(input_text):
    return input_text.lower() 


Dataset avant cleaning:


In [52]:
dataset.head()

,target,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


Appel des fonctions et autres inctructions pour le nettoyage

In [53]:
# suppression des liens URL
dataset['clean_tweet'] = dataset['text'].apply(lambda x: remove_url(x))


In [54]:
# suppression des balises HTML
dataset['clean_tweet'] = dataset['clean_tweet'].apply(lambda x: remove_html_tags(x))

In [55]:
# Suppression des tags (@user)
dataset['clean_tweet'] = np.vectorize(remove_pattern)(dataset['clean_tweet'], "@[\w]*")


In [56]:
# Suppression des non-words (y compris la ponctuation)
dataset['clean_tweet'] = dataset['clean_tweet'].apply(lambda x: remove_non_words(x))


In [57]:
# suppression des mots courts
dataset['clean_tweet'] = dataset['clean_tweet'].apply(lambda x: ' '.join([w for w in x.split() if len(w) >= 3]))


In [58]:
# conversion en minuscule 
dataset['clean_tweet'] = dataset['clean_tweet'].apply(lambda x: x.lower())


In [59]:
# Supprimer stop word (at, the, etc.)
dataset["clean_tweet"] = dataset["clean_tweet"].apply(lambda x : remove_stop_words(x))

In [60]:
# suppression des blancs
dataset['clean_tweet'] = dataset['clean_tweet'].apply(lambda x: remove_gaps(x))


Dataset après le cleaning :

In [61]:
dataset.tail()

,target,text,clean_tweet
1599995,4,Just woke up. Having no school is the best fee...,woke school best feeling ever
1599996,4,TheWDB.com - Very cool to hear old Walt interv...,thewdb com cool hear old walt interviews
1599997,4,Are you ready for your MoJo Makeover? Ask me f...,ready mojo makeover ask details
1599998,4,Happy 38th Birthday to my boo of alll time!!! ...,happy birthday boo alll time tupac amaru shakur
1599999,4,happy #charitytuesday @theNSPCC @SparksCharity...,happy charitytuesday


### Tokenisation, Radicalisation (Stemming) & Lemmatisation


In [72]:
nlp = spacy.load("en_core_web_sm")



stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()


# Function to preprocess text
def preprocess_text(text):
    
    # Tokenize the text using spaCy
    tokens = nlp(text)
    
    # Lemmatize and stem each token, and remove punctuation and stop words
    processed_tokens = [lemmatizer.lemmatize(stemmer.stem(token.text)) for token in tokens]
    
    # Join the tokens back into a string
    processed_text = ' '.join(processed_tokens)
    
    return processed_text


In [74]:
dataset["tokens"] = dataset["clean_tweet"].apply(preprocess_text)

In [75]:
dataset.head()

,target,text,clean_tweet,tokens
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",awww bummer shoulda got david carr third day,awww bummer shoulda got david carr third day
1,0,is upset that he can't update his Facebook by ...,upset update facebook texting might cry result...,upset updat facebook text might cri result sch...
2,0,@Kenichan I dived many times for the ball. Man...,dived many times ball managed save rest bounds,dive mani time ball manag save rest bound
3,0,my whole body feels itchy and like its on fire,whole body feels itchy like fire,whole bodi feel itchi like fire
4,0,"@nationwideclass no, it's not behaving at all....",behaving mad see,behav mad see


### Remplacer les labels 4 par les labels 1
0 --> tweet negatif

1 --> tweet positif

In [77]:
# Remplacer 4 par 1 :
dataset['target'] = dataset['target'].replace(4, 1)


In [78]:
# Dataset apres changement de label target et suppression des colonnes inutiles
dataset.tail()

,target,text,clean_tweet,tokens
1599995,1,Just woke up. Having no school is the best fee...,woke school best feeling ever,woke school best feel ever
1599996,1,TheWDB.com - Very cool to hear old Walt interv...,thewdb com cool hear old walt interviews,thewdb com cool hear old walt interview
1599997,1,Are you ready for your MoJo Makeover? Ask me f...,ready mojo makeover ask details,readi mojo makeov ask detail
1599998,1,Happy 38th Birthday to my boo of alll time!!! ...,happy birthday boo alll time tupac amaru shakur,happi birthday boo alll time tupac amaru shakur
1599999,1,happy #charitytuesday @theNSPCC @SparksCharity...,happy charitytuesday,happi charitytuesday


In [79]:
# supprimer les mots courts de nouveau
dataset['tokens_'] = dataset['tokens'].apply(lambda x: ' '.join([w for w in x.split() if len(w) >= 3]))


dataset.head()


,target,text,clean_tweet,tokens,tokens_
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",awww bummer shoulda got david carr third day,awww bummer shoulda got david carr third day,awww bummer shoulda got david carr third day
1,0,is upset that he can't update his Facebook by ...,upset update facebook texting might cry result...,upset updat facebook text might cri result sch...,upset updat facebook text might cri result sch...
2,0,@Kenichan I dived many times for the ball. Man...,dived many times ball managed save rest bounds,dive mani time ball manag save rest bound,dive mani time ball manag save rest bound
3,0,my whole body feels itchy and like its on fire,whole body feels itchy like fire,whole bodi feel itchi like fire,whole bodi feel itchi like fire
4,0,"@nationwideclass no, it's not behaving at all....",behaving mad see,behav mad see,behav mad see


In [81]:
# suppression des colonnes non utiles
dataset.drop(columns=["text", "clean_tweet", "tokens"], inplace=True)

print("New shape :", dataset.shape)


New shape : (1600000, 2)


Sauvegarder le nouveau dataset preprocessed dans un fichier csv

In [82]:
# Enregistrer le dataset nettoyé dans un nouveau fichier
dataset.to_csv("sentiment140_preprocessed.csv", index=False)


# Annoncer la fin du Nettoyage

In [83]:
import winsound
import time

frequency = 440  
duration = 1000  # 1 seconde

winsound.Beep(frequency, duration)



Veuillez trouver la suite du programme dans le fichier projet2.ipynb